<a href="https://colab.research.google.com/github/manoj-b-22/IDS-in-IoV/blob/main/code/LSTM-inter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
scaler = StandardScaler()
encoder = LabelEncoder()

In [4]:
train = pd.read_csv('/content/drive/MyDrive/train-intervehicle.csv',chunksize=500000)
test = pd.read_csv('/content/drive/MyDrive/test-intervehicle.csv')

In [5]:
model = Sequential()
model.add(LSTM(100,input_shape=(len(test.columns)-1,1)))
model.add(Dense(1))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100)               40800     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________


In [6]:
for chunk in train:
  train_y = chunk.pop('Label')
  train_x = chunk
  train_x = scaler.fit_transform(train_x)
  train_y = encoder.fit_transform(train_y)
  model.fit(train_x,train_y,batch_size=32)

6161/6161 [==============================] - 30s 5ms/step - loss: 7.8231 - accuracy: 0.4928


In [7]:
test_y = test.pop('Label')
test_x = test

In [8]:
test_x = scaler.transform(test_x)
test_y = encoder.transform(test_y)

In [9]:
score,accuracy = model.evaluate(test_x,test_y)
print('Score',score,"Accuracy",accuracy)

79666/79666 [==============================] - 306s 4ms/step - loss: 7.8270 - accuracy: 0.4926
Score 7.8269524574279785 Accuracy 0.4925849735736847
